In [76]:
%env OMP_NUM_THREADS=1

import numpy as np
from scipy import linalg
import os, time

from multiprocessing import Process, Manager, Pipe, cpu_count, Barrier, Array
import ctypes

env: OMP_NUM_THREADS=1


### Input

In [77]:
n = 10
A = np.random.randn(n, n)
L = np.diag(np.arange(10, 10 + n))
A = np.linalg.inv(A) @ L @ A
b = np.random.randn(n)
eps = 1e-6

### Solution

In [78]:
real_x = np.linalg.solve(A, b)
np.linalg.norm(A @ real_x - b)

5.335734195300991e-16

### Rotation function

In [79]:
def rotation(vec, cot):
    assert(len(vec) > len(cot))
    res = np.copy(vec)
    i = 0
    for ctg in cot:
                s = 1 / np.sqrt(ctg ** 2 + 1)
                c = ctg * s
                temp1 = c * vec[i] + s * vec[i + 1]
                temp2 = -s * vec[i] + c * vec[i + 1]
                vec[i] = temp1
                vec[i + 1] = temp2
                i +=1
    return vec        

### GMRES

In [91]:
def gmres(A, b, k, eps):
    n = b.shape[0]
    V = np.zeros((n, k))
    R = np.zeros((k, k)) # H = Q @ R
    e_1_rot = np.zeros_like(b)
    e_1_rot[0] = 1 
    beta = np.linalg.norm(b)
    v = b / beta
    V[:, 0] = v 
    residual = beta
    ctg = []
    m = 0 
    while (residual > eps):
        m += 1
        if m > k - 1:
            print("Number of iterations > k")
            return b, residual
        v = A @ v
        h = np.zeros(m + 1)
        h[:m] = V[:, :m].T @ v
        v = v - V[:, :m] @ h[:m]
        h[m] = np.linalg.norm(v)
        if (h[m] < 1e-14):
            print("dim K = n")
            c = linalg.solve_triangular(R[:m - 1, : m - 1], beta * e_1_rot[:m - 1])
            x = V[:, : m - 1 ] @ c
            print(h[m])
            return x, residual
        v /= h[m]
        V[:, m] = v
        h = rotation(h, ctg)
        ctg.append(h[m - 1]/ h[m])
        h[m - 1:] = rotation(h[m - 1:], [ctg[-1]])
        assert(h[m] < 1e-8)
        R[:m, m - 1] = h[:m]
        e_1_rot[m - 1:] = rotation(e_1_rot[m - 1:], [ctg[-1]])
        residual = beta * abs(e_1_rot[m])
        c = linalg.solve_triangular(R[:m, : m], beta * e_1_rot[:m])
        x = V[:, : m ] @ c
    print('m =', m)
    c = linalg.solve_triangular(R[:m, : m], beta * e_1_rot[:m])
    x = V[:, : m ] @ c
    return x, residual

In [81]:
%%time
x, res = gmres(A, b, 100, eps)

err =  0.03204097783526686
err =  0.005304199069507562
err =  0.00020236611673070544
err =  1.1787675818062815e-05
err =  1.4032524372785912e-06
err =  8.35764829888994e-08
err =  9.860469028466624e-09
m = 7
CPU times: user 7.41 ms, sys: 17.5 ms, total: 24.9 ms
Wall time: 52.7 ms


In [82]:
print(x)
print(res)

[ 0.00257672 -0.00835191  0.11098437 -0.02697429  0.04121384  0.04838488
  0.00336195 -0.02227793  0.01073204  0.06092264]
2.166958227274288e-07


In [83]:
np.linalg.norm(A @ x - b)

2.1669582256856098e-07

### GMRES_parallel

In [92]:
def gmres_parallel(A_shared, b_p, x_shared, V_shared, R_shared, Mem_shared, eps, num_proc, i1, i2, idx, barr, n, k):
    A = np.frombuffer(A_shared, dtype=np.float64).reshape(n, n)
    A_p = A[i1:i2, :] 
    V = np.frombuffer(V_shared, dtype=np.float64).reshape(n, k)
    Mem = np.frombuffer(Mem_shared, dtype=np.float64).reshape(n, num_proc)
    if idx == 0:
        R = np.frombuffer(R_shared, dtype=np.float64).reshape(k, k)
        e_1_rot = np.zeros(n)
        e_1_rot[0] = 1
        ctg = []
        
    x = np.frombuffer(x_shared, dtype=np.float64)
    
    Mem[0, idx] = np.sum(b_p * b_p)
    barr.wait()
    beta = np.sqrt(np.sum((Mem[0, :])))
    residual = beta
    m = 0 # step 0
    V[i1:i2, 0] = b_p / beta
    barr.wait()
    
    while residual > eps:
        m += 1
        if m > (k - 1):
            print("Number of iterations > k")
            return x, residual
        v_p = A_p @ V[:, m - 1]
        h = np.zeros(m + 1)
        h_p = V[i1:i2, :m].T @  v_p
        Mem[:m, idx] = h_p
        barr.wait()
        h[:m] = np.sum(Mem[:m, :], axis=1)
        v_p -=  V[i1:i2, :m] @ h[:m]
        V[i1:i2, m] = v_p
        barr.wait()
        if idx == 0: 
            h[m] = np.linalg.norm(V[:, m])
            if (h[m] < 1e-8):
                print("dim K = n")
                m -= 1
                break;
            V[:, m] /= h[m]
            h = rotation(h, ctg)
            ctg.append(h[m - 1]/ h[m])
            h[m - 1:] = rotation(h[m - 1:], [ctg[-1]])
            assert(h[m] < 1e-8)
            R[:m, m - 1] = h[:m]
            e_1_rot[m - 1:] = rotation(e_1_rot[m - 1:], [ctg[-1]])
            residual = beta * abs(e_1_rot[m])
            Mem[0, 0] = residual
            
        barr.wait()
        residual = Mem[0, 0]
        barr.wait()
    if idx == 0:
        c = linalg.solve_triangular(R[:m, : m], beta * e_1_rot[:m])
        x[:] = (V[:, : m ] @ c)[:]
    barr.wait()
                   
    return x, residual

### 100

In [96]:
times_100 = []
n = 100
K = 50
eps = 1e-6
A_shared = Array(ctypes.c_double, n * n, lock=False)
V_shared = Array(ctypes.c_double, n * K, lock=False)
R_shared = Array(ctypes.c_double, K * K, lock=False)
x_shared = Array(ctypes.c_double, n, lock=False)
b_shared = Array(ctypes.c_double, n, lock=False)
A = np.frombuffer(A_shared, dtype=np.float64).reshape(n,n)
b = np.frombuffer(b_shared, dtype=np.float64)
np.random.seed(0)
P = np.random.randn(n, n)
P = P @ P.T
P += 100 * np.eye(n)
A[:] = P[:]
b = np.random.randn(n)

In [98]:
#gmres
t = time.time()
x, res = gmres(A, b, K, eps)
t = time.time() - t
times_100.append(t)
print("num_proc = 1")
assert(np.linalg.norm(A.dot(x) - b) < eps)
print("t = ", t)
t_base = t

#parallel gmres
for num_proc in range(2, cpu_count() + 1, 1):
    barr = Barrier(num_proc)
    Mem_shared = Array(ctypes.c_double, n * num_proc, lock=False)
    block_size = n // num_proc
    block_size += bool(n % num_proc)
    i_pos = [min(i * block_size, n) for i in range(num_proc + 1)]
    proc_list = [Process(target=gmres_parallel, 
                        args=(A_shared, b[i_pos[i]:i_pos[i + 1]], x_shared, V_shared, R_shared, Mem_shared, eps, num_proc,
                              i_pos[i], i_pos[i + 1], i, barr, n, K)) for i in range(num_proc)]
    t = time.time()

    for proc in proc_list:
        proc.start()

    for proc in proc_list:
        proc.join()

    t = time.time() - t
    times_100.append(t)
    x = np.frombuffer(x_shared, dtype=np.float64)
    print("num_proc = ", num_proc)
    #assert(np.linalg.norm(A.dot(x) - b) < eps)
    print("speed = ", t_base / t)
    del Mem_shared
    
del V_shared
del R_shared
del x_shared

m = 16
num_proc = 1
t =  0.010756731033325195
num_proc =  2
speed =  0.04547896709914207
num_proc =  3
speed =  0.1417463100152689
num_proc =  4
speed =  0.1222368224982728
num_proc =  5
speed =  0.09565191581562885
num_proc =  6
speed =  0.08017251030210627
num_proc =  7
speed =  0.06753809374826915
num_proc =  8
speed =  0.06232981461449513


### 1000

In [99]:
times_1000 = []
n = 1000
K = 50
eps = 1e-6
A_shared = Array(ctypes.c_double, n * n, lock=False)
V_shared = Array(ctypes.c_double, n * K, lock=False)
R_shared = Array(ctypes.c_double, K * K, lock=False)
x_shared = Array(ctypes.c_double, n, lock=False)
A = np.frombuffer(A_shared, dtype=np.float64).reshape(n,n)
b = np.frombuffer(b_shared, dtype=np.float64)
np.random.seed(0)
P = np.random.randn(n, n)
P = P @ P.T
P += 200 * np.eye(n)
A[:] = P[:]
b = np.random.randn(n)

In [101]:
#gmres
t = time.time()
x, res = gmres(A, b, K, eps)
t = time.time() - t
times_1000.append(t)
print("num_proc = 1")
assert(np.linalg.norm(A.dot(x) - b) < eps)
print("t = ", t)
t_base = t

#parallel gmres
for num_proc in range(2, cpu_count() + 1, 1):
    barr = Barrier(num_proc)
    Mem_shared = Array(ctypes.c_double, n * num_proc, lock=False)
    block_size = n // num_proc
    block_size += bool(n % num_proc)
    i_pos = [min(i * block_size, n) for i in range(num_proc + 1)]
    proc_list = [Process(target=gmres_parallel, 
                        args=(A_shared, b[i_pos[i]:i_pos[i + 1]], x_shared, V_shared, R_shared, Mem_shared, eps, num_proc,
                              i_pos[i], i_pos[i + 1], i, barr, n, K)) for i in range(num_proc)]
    t = time.time()

    for proc in proc_list:
        proc.start()

    for proc in proc_list:
        proc.join()

    t = time.time() - t
    times_1000.append(t)
    x = np.frombuffer(x_shared, dtype=np.float64)
    print("num_proc = ", num_proc)
    #assert(np.linalg.norm(A.dot(x) - b) < eps)
    print("speed = ", t_base / t)
    del Mem_shared
    
del V_shared
del R_shared
del x_shared


m = 39
num_proc = 1
t =  0.04167819023132324
num_proc =  2
speed =  0.34653506563532055
num_proc =  3
speed =  0.3994757781632126
num_proc =  4
speed =  0.43175043036486543
num_proc =  5
speed =  0.3531121605953242
num_proc =  6
speed =  0.27720936870649054
num_proc =  7
speed =  0.1915825166747401
num_proc =  8
speed =  0.18997912317327767


### Numba

In [104]:
import numba

In [138]:
n = 10
A = np.random.randn(n, n)
L = np.diag(np.arange(20, 20 + n))
A = np.linalg.inv(A) @ L @ A
b = np.random.randn(n)
eps = 1e-6

real_x = np.linalg.solve(A, b)
np.linalg.norm(A @ real_x - b)

7.065416064076988e-16

In [152]:
@numba.jit(nopython=True)
def rotation(vec, cot):
    assert(len(vec) > len(cot))
    res = np.copy(vec)
    i = 0
    for ctg in cot:
                s = 1 / np.sqrt(ctg ** 2 + 1)
                c = ctg * s
                temp1 = c * vec[i] + s * vec[i + 1]
                temp2 = -s * vec[i] + c * vec[i + 1]
                vec[i] = temp1
                vec[i + 1] = temp2
                i +=1
    return vec  

def gmres(A, b, k, eps):
    n = b.shape[0]
    V = np.zeros((n, k))
    R = np.zeros((k, k)) # H = Q @ R
    e_1_rot = np.zeros_like(b)
    e_1_rot[0] = 1
    m = 0 # step 0
    beta = np.linalg.norm(b)
    v = b / beta
    V[:, 0] = v 
    residual = beta
    ctg = []
    while (residual > eps):
        m += 1 #step m
        if m > k - 1:
            print("Number of iterations > k")
            return b, residual
        v = A @ v
        h = np.zeros(m + 1)
        h[:m] = V[:, :m].T @ v
        v = v - V[:, :m] @ h[:m]
        h[m] = np.linalg.norm(v)
        if (h[m] < 1e-14):
            print("dim K = n")
            c = linalg.solve_triangular(R[:m - 1, : m - 1], beta * e_1_rot[:m - 1])
            x = V[:, : m - 1 ] @ c
            print(h[m])
            return x, residual
        v /= h[m]
        V[:, m] = v
        h = rotation(h, ctg)
        ctg.append(h[m - 1]/ h[m])
        h[m - 1:] = rotation(h[m - 1:], [ctg[-1]])
        assert(h[m] < 1e-8)
        R[:m, m - 1] = h[:m]
        e_1_rot[m - 1:] = rotation(e_1_rot[m - 1:], [ctg[-1]])
        residual = beta * abs(e_1_rot[m])
        c = linalg.solve_triangular(R[:m, : m], beta * e_1_rot[:m])
        x = V[:, : m ] @ c
        #print("err = ", np.linalg.norm(x - real_x))
    print('m =', m)
    c = linalg.solve_triangular(R[:m, : m], beta * e_1_rot[:m])
    x = V[:, : m ] @ c
    return x, residual


In [156]:
%%time
x, res = gmres(A, b, 100, eps)

m = 6
CPU times: user 2.81 ms, sys: 2.01 ms, total: 4.82 ms
Wall time: 3.23 ms
